<a href="https://colab.research.google.com/github/dougfeltrim/Gemini-test/blob/main/Gemini_software.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
from google.colab import userdata

os.environ['GOOGLE_API_KEY'] = userdata.get('GOOGLE_API_KEY')

In [ ]:
# prompt: Gere uma empresa de software usando agentes, preciso que ao final consiga gerar um projeto completo e pronto para o cliente, no final preciso que imprima o codigo final e sua documentacao

import os
from google.colab import userdata
from google import genai
from google.genai import types

os.environ['GOOGLE_API_KEY'] = userdata.get('GOOGLE_API_KEY')

# Remove the explicit api_key argument to use the environment variable
client = genai.Client()

class Agent:
    def __init__(self, name, system_instruction):
        self.name = name
        self.system_instruction = system_instruction
        self.history = []

    def generate_response(self, prompt):
        response = client.models.generate_content(
            model="gemini-2.0-flash",
            config=types.GenerateContentConfig(
                system_instruction=self.system_instruction,
                max_output_tokens=1024,
                temperature=0.7 # Lower temperature for more focused responses
            ),
            contents=prompt
        )
        self.history.append({"prompt": prompt, "response": response.text})
        return response.text

class SoftwareCompany:
    def __init__(self):
        self.agents = {
            "Product Manager": Agent("Gerente de Produto", "Você define os requisitos do software e a visão do produto."),
            "Software Architect": Agent("Arquiteto de Software", "Você projeta a arquitetura do software e as tecnologias a serem usadas."),
            "Developer": Agent("Desenvolvedor", "Você escreve o código real do software."),
            "Technical Writer": Agent("Redator Técnico", "Você escreve a documentação do software."),
            "Project Synthesizer": Agent("Sintetizador de Projeto", "Você consolida o trabalho de todos os agentes em um projeto final. Crie o respitorio do projeto após a consolidaçao")
        }

    def generate_project(self, project_description, rounds=5):
        current_task = f"Crie um projeto de software baseado na seguinte descrição: {project_description}"
        project_components = {}

        for i in range(rounds):
            print(f"\n--- Rodada {i+1} ---")
            responses = {}
            for role, agent in self.agents.items():
                if role != "Project Synthesizer": # Synthesizer acts last
                    print(f"{agent.name} pensando sobre: {current_task}...")
                    response = agent.generate_response(current_task)
                    responses[role] = response
                    print(f"{agent.name}: {response}")

            # Synthesizer consolidates the work
            synthesis_prompt = "Consolide o seguinte trabalho para avançar no projeto:\n"
            for role, resp in responses.items():
                synthesis_prompt += f"Do {role}: {resp}\n"

            print("\nSintetizador de Projeto consolidando...")
            current_task = self.agents["Project Synthesizer"].generate_response(synthesis_prompt)
            print(f"Sintetizador de Projeto (Síntese): {current_task}")

            # Assuming in later rounds, agents start generating specific components
            # This is a simplified example; in a real scenario, you'd need more sophisticated parsing
            if i >= rounds - 2: # In the last two rounds, focus on generating code and documentation
                code_prompt = "Gere o código Python para o projeto baseado na tarefa atual:"
                code_response = self.agents["Developer"].generate_response(code_prompt + "\n" + current_task)
                project_components["code"] = code_response

                doc_prompt = "Gere a documentação técnica para o projeto baseado na tarefa atual e código gerado:"
                doc_response = self.agents["Technical Writer"].generate_response(doc_prompt + "\n" + current_task + "\nCódigo:\n" + code_response)
                project_components["documentation"] = doc_response


        # Final consolidation by the Project Synthesizer
        final_synthesis_prompt = "Com base em todo o trabalho realizado, gere o projeto final pronto para o cliente, incluindo código e documentação:\n"
        if "code" in project_components:
             final_synthesis_prompt += f"Código Gerado:\n{project_components['code']}\n"
        if "documentation" in project_components:
             final_synthesis_prompt += f"Documentação Gerada:\n{project_components['documentation']}\n"
        final_project_summary = self.agents["Project Synthesizer"].generate_response(final_synthesis_prompt)


        return project_components.get("code", "Não foi possível gerar o código."), project_components.get("documentation", "Não foi possível gerar a documentação.")

# Exemplo de uso da empresa de software
company = SoftwareCompany()
project_description = "Desenvolver um script de calculadora para ser executado no terminal"
final_code, final_documentation = company.generate_project(project_description, rounds=1) # More rounds for a more complete project


--- Rodada 1 ---
Gerente de Produto pensando sobre: Crie um projeto de software baseado na seguinte descrição: Desenvolver um script de calculadora para ser executado no terminal...
Gerente de Produto: ## Projeto de Software: Script de Calculadora para Terminal

**1. Visão Geral:**

Este projeto visa desenvolver um script de calculadora interativa que será executado no terminal. O script permitirá que os usuários realizem operações aritméticas básicas (adição, subtração, multiplicação, divisão) e, potencialmente, funções mais avançadas, diretamente da linha de comando. A calculadora será fácil de usar, intuitiva e fornecerá resultados precisos.

**2. Objetivos:**

*   **Funcionalidade:** Implementar as operações aritméticas básicas (adição, subtração, multiplicação, divisão).
*   **Usabilidade:** Criar uma interface de usuário simples e intuitiva no terminal.
*   **Precisão:** Garantir que os cálculos sejam realizados com precisão.
*   **Robustez:** Lidar com entradas inválidas de for

In [ ]:
print("\n--- Projeto Final ---")
print("\nCódigo:")
print(str(final_code))


--- Projeto Final ---

Código:
```python
import math
import re

class InvalidExpressionError(Exception):
    """Exceção para expressões inválidas."""
    pass

def evaluate_expression(expression):
    """
    Avalia uma expressão matemática.

    Args:
        expression: A expressão matemática como uma string.

    Returns:
        O resultado da expressão como um float.

    Raises:
        InvalidExpressionError: Se a expressão for inválida.
        ZeroDivisionError: Se houver uma divisão por zero.
    """

    try:
        # Remove espaços em branco
        expression = expression.replace(" ", "")

        # Validação básica para evitar eval() perigoso.  Mais validação é feita posteriormente.
        if not re.match(r"^[\d\+\-\*\/\(\)\.\s]+$", expression):
            raise InvalidExpressionError("Expressão contém caracteres inválidos.")

        # Substitui sqrt por math.sqrt para avaliação segura
        expression = re.sub(r"sqrt\((.*?)\)", r"math.sqrt(\1)", expression)

     

In [ ]:
print("\nDocumentação:")
print(str(final_documentation))


Documentação:
OK. Aqui está a documentação técnica para o projeto de script de calculadora de terminal, com base nas informações fornecidas.

# Documentação Técnica: Script de Calculadora de Terminal

## 1. Introdução

Este documento fornece uma descrição técnica detalhada do script de calculadora de terminal. Ele aborda a arquitetura, o design, a implementação e o uso do script.  O objetivo é fornecer informações suficientes para desenvolvedores que desejam entender, modificar ou estender a funcionalidade da calculadora.

## 2. Visão Geral

O script de calculadora de terminal é uma aplicação de linha de comando que permite aos usuários realizar operações aritméticas diretamente no terminal. Ele oferece uma interface simples e intuitiva para inserir expressões matemáticas e exibir os resultados. O script foi projetado com foco na precisão, robustez, manutenibilidade e extensibilidade.

## 3. Arquitetura

O script segue uma arquitetura modular, dividida em três componentes principais:
